# Output Parsers for LLM Input / Output with LangChain

## Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install -qq langchain==0.3.11
!pip install -qq langchain-openai==0.2.12
!pip install -qq langchain-community==0.3.11

In [ ]:
!pip install -qq pandas

## Enter API Tokens

#### Enter your Open AI Key here

You can get the key from [here](https://platform.openai.com/api-keys) after creating an account or signing in

In [ ]:
# from getpass import getpass

# OPENAI_KEY = getpass('Please enter your Open AI API Key here: ')

In [ ]:
# import os

# os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

## Setup necessary system environment variables

## Chat Models and LLMs

Large Language Models (LLMs) are a core component of LangChain. LangChain does not implement or build its own LLMs. It provides a standard API for interacting with almost every LLM out there.

There are lots of LLM providers (OpenAI, Hugging Face, etc) - the LLM class is designed to provide a standard interface for all of them.

## Accessing Commercial LLMs like ChatGPT

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Output Parsers
Output parsers are essential in Langchain for structuring the responses from language models. Below, we will discuss the role of output parsers and include examples using Langchain's specific parser types: PydanticOutputParser, JsonOutputParser, and CommaSeparatedListOutputParser.

- **Pydantic parser:**
  - This parser allows the specification of an arbitrary Pydantic Model to query LLMs for outputs matching that schema. Pydantic's BaseModel functions similarly to a Python dataclass but includes type checking and coercion.

- **JSON parser:**
  - Users can specify an arbitrary JSON schema with this parser to ensure outputs from LLMs adhere to that schema. Pydantic can also be used to declare your data model here.

- **CSV parser:**
  - Useful for outputs requiring a list of items separated by commas. This parser facilitates the extraction of comma-separated values from model outputs.


### 1 -> PydanticOutputParser

This output parser allows users to specify an arbitrary Pydantic Model and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are non-deterministic! You'll have to use an LLM with sufficient capacity to generate well-formed responses.

Use Pydantic to declare your data model. Pydantic's BaseModel is like a Python dataclass, but with actual type checking + coercion.

In [ ]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import List

# Define your desired data structure - like a python data class.
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field(description="One line conclusion of the topic asked by the user")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=QueryResponse)
parser

In [ ]:
# langchain pre-generated output response formatting instructions
print(parser.get_format_instructions())

In [ ]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Answer the user query and generate the response based on the following formatting instructions

             Format Instructions:
             {format_instructions}

             Query:
             {query}
            """

prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | chatgpt | parser

response = chain.invoke({"query": "What is the capital of France?"})
print(response)

In [ ]:
question = "Tell me about Commercial Real Estate"
response = chain.invoke({"query": question})

In [ ]:
print(response.description)

In [ ]:
print(response.pros)

In [ ]:
response.dict()

In [ ]:
for k,v in response.dict().items():
    print(f"{k}:\n{v}\n")

### JsonOutputParser

This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.

Keep in mind that large language models are non-deterministic! You'll have to use an LLM with sufficient capacity to generate well-formed responses.

It is recommended use Pydantic to declare your data model.


In [ ]:
from typing import List

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure - like a python data class.
class QueryResponse(BaseModel):
    description: str = Field(description="A brief description of the topic asked by the user")
    pros: str = Field(description="3 bullet points showing the pros of the topic asked by the user")
    cons: str = Field(description="3 bullet points showing the cons of the topic asked by the user")
    conclusion: str = Field(description="One line conclusion of the topic asked by the user")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=QueryResponse)
parser

In [ ]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Answer the user query and generate the response based on the following formatting instructions

             Format Instructions:
             {format_instructions}

             Query:
             {query}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(prompt)

In [ ]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
              |
            chatgpt
              |
            parser)
chain

In [ ]:
topic_queries = [
    "Tell me about commercial real estate",
    "Tell me about Generative AI"
]

topic_queries_formatted = [{"query": topic}
                    for topic in topic_queries]
topic_queries_formatted

In [ ]:
responses = chain.map().invoke(topic_queries_formatted)

In [ ]:
responses[0], type(responses[0])

In [ ]:
import pandas as pd

df = pd.DataFrame(responses)
df

In [ ]:
for response in responses:
  for k,v in response.items():
    print(f"{k}:\n{v}\n")
  print('-----')

### CommaSeparatedListOutputParser

This output parser can be used when you want to return a list of comma-separated items.

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
format_instructions = output_parser.get_format_instructions()

# And a query intented to prompt a language model to populate the data structure.
prompt_txt = """
             Create a list of 5 different ways in which Generative AI can be used

             Output format instructions:
             {format_instructions}
             """

prompt = PromptTemplate.from_template(template=prompt_txt)

prompt = PromptTemplate(
    template=prompt_txt,
    partial_variables={"format_instructions": format_instructions},
)
print(prompt)

In [ ]:
# create a simple LLM Chain - more on this later
llm_chain = (prompt
              |
            chatgpt
              |
            output_parser)

# run the chain
response = llm_chain.invoke({})

In [ ]:
print(response)

In [ ]:
response

In [ ]:
type(response)